# Try numba

Arrow doesn't seem to work in Numba. Float Numpy does, but by default I get an object array out which is weird!

In [1]:
import numba

In [8]:
import pandas as pd
import polars as pl
from humanfriendly import format_size, format_number
from simpler_mpl import set_commas, set_common_mpl_styles
import matplotlib.pyplot as plt
import seaborn as sns

#%load_ext autoreload
#%autoreload 2

display(f"Pandas {pd.__version__}, Polars {pl.__version__}")

#from cell_profiler import cell_profiler as cp
#%start_cell_profiler
%load_ext ipython_memory_usage
%imu_start


'Pandas 2.1.3, Polars 0.19.17'

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


'IPython Memory Usage started'

In [8] used 0.1 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26945.9 MiB


In [3]:
dfpda = pd.read_parquet("../test_result_2021on.parquet", 
                        dtype_backend="pyarrow")

In [4]:
dfpda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82012245 entries, 0 to 82012244
Data columns (total 14 columns):
 #   Column             Dtype                 
---  ------             -----                 
 0   test_id            int64[pyarrow]        
 1   vehicle_id         int64[pyarrow]        
 2   test_date          timestamp[us][pyarrow]
 3   test_class_id      int64[pyarrow]        
 4   test_type          large_string[pyarrow] 
 5   test_result        large_string[pyarrow] 
 6   test_mileage       int64[pyarrow]        
 7   postcode_area      large_string[pyarrow] 
 8   make               large_string[pyarrow] 
 9   model              large_string[pyarrow] 
 10  colour             large_string[pyarrow] 
 11  fuel_type          large_string[pyarrow] 
 12  cylinder_capacity  int64[pyarrow]        
 13  first_use_date     timestamp[us][pyarrow]
dtypes: int64[pyarrow](5), large_string[pyarrow](7), timestamp[us][pyarrow](2)
memory usage: 10.4 GB


In [5]:
dfpdn = pd.read_parquet("../test_result_2021on.parquet", 
                        dtype_backend="numpy_nullable")

In [9]:
ser_mileage_pdn = dfpdn['test_mileage'].fillna(0).to_numpy()
type(ser_mileage_pdn)
# +0.7GB

numpy.ndarray

In [9] used 1.6 MiB RAM in 2.41s (system mean cpu 10%, single max cpu 100%), peaked 702.4 MiB above final usage, current RAM usage now 26947.5 MiB


In [10]:
ser_mileage_pdn.dtype # !!! why object?!

dtype('O')

In [10] used 0.1 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26947.6 MiB


In [11]:
ser_mileage_pdn = ser_mileage_pdn.astype('float')
ser_mileage_pdn.dtype

dtype('float64')

In [11] used -2492.3 MiB RAM in 2.67s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 24455.3 MiB


In [ ]:
#dfpda['test_mileage'].to_numpy() # feather
# used 3157.7 MiB RAM in 2.68s

In [12]:
ser_mileage_pda = dfpda['test_mileage'].fillna(0).values
type(ser_mileage_pda)

pandas.core.arrays.arrow.array.ArrowExtensionArray

In [12] used 635.5 MiB RAM in 0.46s (system mean cpu 12%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 25090.8 MiB


In [13]:
def do_thing(ser):
    # given ser, loop and find max
    mx = 0
    for idx in range(ser.shape[0]):
        value = ser[idx]
        if value > mx:
            mx = value
    return mx

In [13] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25090.8 MiB


In [14]:
do_thing(ser_mileage_pdn)

999999.0

In [14] used 0.0 MiB RAM in 5.18s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 25090.8 MiB


In [15]:
do_thing(ser_mileage_pda[:100000]) # crazy slow if not limited!

997046

In [15] used 0.0 MiB RAM in 0.31s (system mean cpu 8%, single max cpu 29%), peaked 0.0 MiB above final usage, current RAM usage now 25090.8 MiB


In [16]:
from numba import njit
do_thing_njit = njit(do_thing)

In [16] used 7.2 MiB RAM in 0.11s (system mean cpu 4%, single max cpu 9%), peaked 0.0 MiB above final usage, current RAM usage now 25098.0 MiB


In [17]:
do_thing_njit(ser_mileage_pdn)

999999.0

In [17] used 28.4 MiB RAM in 0.43s (system mean cpu 10%, single max cpu 94%), peaked 0.0 MiB above final usage, current RAM usage now 25126.4 MiB


In [19]:
#- argument 0: Cannot determine Numba type of <class 'pandas.core.arrays.arrow.array.ArrowExtensionArray'>
#do_thing_njit(ser_mileage_pda)

In [19] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 25133.8 MiB


# Bit unsure about all of this!

In [ ]:
import numpy as np

In [ ]:
%timeit ser_mileage_pdn.mean()

In [ ]:
%timeit np.nanmean(ser_mileage_pdn)

In [ ]:
%timeit dfpdn['test_mileage'].fillna(0).mean()

In [ ]:
dfpdn['test_mileage'].fillna(0).mean()

In [ ]:
dfpdn['test_mileage'].mean()

In [ ]:
%timeit dfpdn['test_mileage'].mean()

In [ ]:
%timeit dfpda['test_mileage'].fillna(0).mean()

# Trying sane item

TODO try to balance up speed timings on mean?

In [ ]:
ser_pdn = pd.Series(np.array([0.1] * 1_000_000))
assert ser_pdn.dtype == np.dtype('float64')

In [ ]:
ser_pda = ser_pdn.astype('float64[pyarrow]')
#assert ser_pda.dtype == np.dtype('double[pyarrow]')

In [ ]:
ser_pl = pl.from_pandas(ser)

In [ ]:
ser_pdn.mean()

In [ ]:
ser_pda.mean()

In [ ]:
ser_pl.mean()